# 1. Definición del Problema

Comprender qué tipo de vino debo consumir en consideración del meridaje y mi perfil sociodemográfico.

# 2. Obtención del conjunto de datos

In [3]:
!pip install selenium
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Obtención de url de productos de la CAV - URL guardadas en CSV

import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup as soup

# Obtener la data de los vinos desde la página de la CAV/tienda
data = []
driver = webdriver.Chrome()

for x in range(50):
    url = f"https://cav.cl/tienda?q=&hPP=20&idx=products&p={x}&fR%5Bfamily.name%5D%5B0%5D=Vinos&is_v=1"
    driver.get(url)
    
    try:
        age_verification_popup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'modalAge'))
        )
        yes_button = age_verification_popup.find_element(By.XPATH, ".//button[contains(., 'SI')]")
        yes_button.click()
    except (NoSuchElementException, TimeoutException):
        pass

    page_soup = soup(driver.page_source, 'html.parser')
    
    info = []
    for article in page_soup.find_all("article"):
        id = article.attrs.get("data-id")
        sku = article.attrs.get("data-sku")
        name = article.attrs.get("data-name")
        brand = article.attrs.get("data-brand")
        category = article.attrs.get("data-category")
        variant = article.attrs.get("data-variant")
        ref = article.find("a").attrs.get("href")
        info.append([id, sku, name, brand, category, variant, ref])
    
    data.append(info)
    
    # Guardar datos en un archivo CSV después de cada iteración
    df_partial = pd.DataFrame(info, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])
    df_partial.drop_duplicates("id", inplace=True)
    df_partial.to_csv(f"data//raw//vinos_parte_{x}.csv", sep=',', mode='w', index=False)

driver.quit()

# Reshape la data
reshaped_data = [item for sublist in data for item in sublist]

# Crear el DataFrame
df = pd.DataFrame(reshaped_data, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])

# Eliminar los duplicados
df.drop_duplicates("id", inplace=True)

# Guardar CSV de los vinos encontrados
df.to_csv("data//raw//vinos.csv", sep=',', mode='w', index=False)

# Borrar los pack de vinos
df_filtered = df.drop(axis=0, index=df.index[:8])

# Completar la URL en la columna 'ref'
df_filtered['ref'] = 'https://cav.cl/' + df_filtered['ref'].astype(str)

# Guardar CSV solo vinos
df_filtered.to_csv("url_products.csv", sep=',', mode='w', index=False)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/vscode/.cache/selenium/chrome/linux64/119.0.6045.105/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x555ac23695e3 <unknown>
#1 0x555ac202c0b7 <unknown>
#2 0x555ac2062e55 <unknown>
#3 0x555ac205fb81 <unknown>
#4 0x555ac20aa47f <unknown>
#5 0x555ac20a0cc3 <unknown>
#6 0x555ac206c0e4 <unknown>
#7 0x555ac206d0ae <unknown>
#8 0x555ac232fce1 <unknown>
#9 0x555ac2333b7e <unknown>
#10 0x555ac231d4b5 <unknown>
#11 0x555ac23347d6 <unknown>
#12 0x555ac2300dbf <unknown>
#13 0x555ac2357748 <unknown>
#14 0x555ac2357917 <unknown>
#15 0x555ac2368773 <unknown>
#16 0x7faa05e3dea7 start_thread


In [ ]:
#  Obtención de detallada de cada producto desde la URL scrapeada 

import requests
from bs4 import BeautifulSoup
import csv

# Leer las URL desde el archivo CSV
url_list = []

with open('product_url_list.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        url = row['URL']
        url_list.append(url)

# Crear una lista para almacenar los datos de cada producto
all_product_data = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Acceder a los atributos 'data-*' de la etiqueta 'section'
    product_section = soup.select_one('section.c-product')

    product_id = product_section['data-id']
    product_sku = product_section['data-sku']
    product_name = product_section['data-name']
    product_brand = product_section['data-brand']
    product_category = product_section['data-category']
    product_variant = product_section['data-variant']
    product_price = product_section['data-price']

    # Acceder a la ficha técnica
    wine_table = soup.find('table', class_='table table-striped')

    # Inicializar variables para los campos de la tabla
    wine_data = {
        "vinyard": "",
        "strain": "",
        "harvest": "",
        "origin": "",
        "line": "",
        "format": "",
        "type": "",
        "alcohol_degrees": "",
        "blend": "",
        "guard": "",
        "pairing": ""
    }

    if wine_table:
        # Obtener todas las filas de la tabla
        rows = wine_table.find_all('tr')

        for row in rows:
            header = row.find('th')
            data = row.find('td')

            if header and data:
                header_text = header.text.strip()
                data_text = data.text.strip()

                # Asignar los datos a las categorías correspondientes
                if header_text in wine_data:
                    wine_data[header_text] = data_text

    # Acceder al puntaje y calificaciones
    score_section = product_section.select_one('.c-product__score')

    if score_section is not None:
        lacav_score_element = score_section.select_one('.o-score--lacav .o-score__points')
        if lacav_score_element is not None:
            lacav_score = lacav_score_element.text
        else:
            lacav_score = "No score available"
    else:
        lacav_score = "No score available"

    # description
    description = soup.select_one('.c-product__body p').text.strip()

    # Información de precios y stock
    price_info = soup.select_one('.c-product__price')
    socio_price = price_info.select('p.o-text--price-store')[0].text
    normal_price = price_info.select('p.o-text--small')[0].text
    stock = price_info.select('p.o-text--small')[1].text

    # Acceder a la image_url
    product_image = soup.select_one('div.c-product__thumb img')['src']

    # Acceder a la sección de percepción
    sheet_body = soup.select_one('div.c-sheet__body')
    dl_elements = sheet_body.find_all('dl', class_='c-sheet__feature--chart') if sheet_body else []

    perception_data = {}

    for dl in dl_elements:
        # Acceder al contenido de c-sheet__signify
        signify = dl.select_one('dd.c-sheet__signify').text.strip()

        # Acceder al contenido de c-chart__track
        chart_track = dl.select_one('div.c-chart__track')
        chart_class_element = chart_track.find('div', class_='c-chart__percent') if chart_track else None
        if chart_class_element and 'class' in chart_class_element.attrs:
            chart_class = chart_class_element['class'][1]

        # Mapear chart_class a su valor correspondiente
        chart_class_map = {
            'c-chart__percent--0': 0,
            'c-chart__percent--20': 0.2,
            'c-chart__percent--40': 0.4,
            'c-chart__percent--60': 0.6,
            'c-chart__percent--80': 0.8,
            'c-chart__percent--100': 1
        }
        chart_value = chart_class_map.get(chart_class, None)

        perception_data[signify] = chart_value

    # Crear un diccionario con la información
    product_data = {
        "url_product": url,
        "product_internal_id": product_id,
        "product_code": product_sku,
        "pairing": pairing,
        "product_brand": product_brand,
        "product_category": product_category,
        "variant": product_variant,
        "price": product_price,
        "cav_points": lacav_score,
        "description": description,
        "member_price": socio_price,
        "normal_price": normal_price,
        "stock": stock,
        "image_url": product_image,
        "vinyard": wine_data["vinyard"],
        "strain": wine_data["strain"],
        "harvest": wine_data["harvest"],
        "origin": wine_data["origin"],
        "line": wine_data["line"],
        "format": wine_data["format"],
        "type": wine_data["type"],
        "alcohol_degrees": wine_data["alcohol_degrees"],
        "blend": wine_data["blend"],
        "guard": wine_data["guard"],
        "pairing": wine_data["pairing"],
        **perception_data  # Agregar datos de percepción
    }

    all_product_data.append(product_data)

# Resto del código sin cambios

fieldnames = [
    "url_product",
    "product_internal_id",
    "product_code",
    "pairing",
    "product_brand",
    "product_category",
    "variant",
    "price",
    "cav_points",
    "description",
    "member_price",
    "normal_price",
    "stock",
    "image_url",
    "vinyard",
    "strain",
    "harvest",
    "origin",
    "line",
    "format",
    "type",
    "alcohol_degrees",
    "blend",
    "guard",
    "pairing",
    "Cuerpo",
    "Frescor",
    "Dulzura",
    "Astringencia"
]


# guardr los datos en un archivo CSV
csv_filename = "productos.csv"

with open(csv_filename, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for product_data in all_product_data:
        writer.writerow(product_data)



# 3. Almacenamiento de la información

In [1]:
import pandas as pd

productos = pd.read_csv('../data/raw/productos.csv') 
meridaje = pd.read_csv('../data/raw/meridaje.csv')
ranking = pd.read_csv('../data/raw/brand_shopper_rank.csv') 

In [2]:
import sqlite3

conn = sqlite3.connect("../data/wine_db.db")
productos.to_sql("productos", conn, if_exists="replace", index=False)
meridaje.to_sql("meridaje", conn, if_exists="replace", index=False)
ranking.to_sql("ranking", conn, if_exists="replace", index=False)

3604

In [5]:
query = """
SELECT *
FROM productos
JOIN meridaje
ON productos.variant = meridaje.variant
LEFT JOIN ranking
ON productos.product_brand = ranking.brand
;
"""

df = pd.read_sql_query(query, conn)

# Eliminar filas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

# Renombrar columnas 
df = df.rename(columns={'Cuerpo':'body','Astringencia':'astringency','Frescor':'freshness','Dulzura':'sweetness'}) 

# Crear tabla "wine_database"
df.to_sql("wine_database", conn, if_exists="replace", index=False)

conn.close()

In [6]:
df.to_csv('wine_database.csv', index=False)

# 4. Análisis descriptivo

In [7]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/wine_db.db')

# Leer tabla "wine_database"
query = "SELECT * FROM wine_database;"
df = pd.read_sql_query(query, conn)

conn.close()

In [8]:
df.columns

Index(['url_product', 'product_internal_id', 'product_code', 'product_name',
       'product_brand', 'product_category', 'variant', 'price', 'cav_points',
       'description', 'member_price', 'normal_price', 'stock', 'image_url',
       'vinyard', 'strain', 'harvest', 'origin', 'line', 'format', 'type',
       'alcohol_degrees', 'blend', 'guard', 'pairing_cav', 'body', 'freshness',
       'sweetness', 'astringency', 'pairing', 'brand', 'gender', 'age', 'ses',
       'rank'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44500 entries, 0 to 44499
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   url_product          44500 non-null  object 
 1   product_internal_id  44500 non-null  int64  
 2   product_code         44500 non-null  int64  
 3   product_name         44500 non-null  object 
 4   product_brand        44500 non-null  object 
 5   product_category     44500 non-null  object 
 6   variant              44500 non-null  object 
 7   price                44500 non-null  int64  
 8   cav_points           44500 non-null  object 
 9   description          25755 non-null  object 
 10  member_price         44500 non-null  object 
 11  normal_price         44500 non-null  object 
 12  stock                44500 non-null  object 
 13  image_url            44500 non-null  object 
 14  vinyard              44500 non-null  object 
 15  strain               44500 non-null 

# 5. EDA (Exploratory data analysis)

In [23]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/wine_db.db')

# Leer tabla "wine_database"
query = "SELECT * FROM wine_database;"
df = pd.read_sql_query(query, conn)

conn.close()

In [24]:
df.columns

Index(['url_product', 'product_internal_id', 'product_code', 'product_name',
       'product_brand', 'product_category', 'variant', 'price', 'cav_points',
       'description', 'member_price', 'normal_price', 'stock', 'image_url',
       'vinyard', 'strain', 'harvest', 'origin', 'line', 'format', 'type',
       'alcohol_degrees', 'blend', 'guard', 'pairing_cav', 'body', 'freshness',
       'sweetness', 'astringency', 'pairing', 'brand', 'gender', 'age', 'ses',
       'rank'],
      dtype='object')

In [232]:
df.shape

(44500, 2)

# 6. Test 1: Modelado: Input de meridaje -> Recomendación de vino

In [25]:
df = df.dropna()

In [26]:
df = df.drop_duplicates(subset='product_name')

In [13]:
df

,url_product,product_internal_id,product_code,product_name,product_brand,product_category,variant,price,cav_points,description,...,body,freshness,sweetness,astringency,pairing,brand,gender,age,ses,rank
75,https://www.cav.cl/tienda/producto/aila-heaven...,12785,28846,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,Aila,Tinto,Cabernet Sauvignon,9990,91,"Rojo violeta brillante, de ribetes rubíes. Aro...",...,1.0,1.0,0.4,0.4,Carne de Res,Aila,HOMBRE,18-24,ABC1,15.0
76,https://www.cav.cl/tienda/producto/aila-heaven...,12785,28846,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,Aila,Tinto,Cabernet Sauvignon,9990,91,"Rojo violeta brillante, de ribetes rubíes. Aro...",...,1.0,1.0,0.4,0.4,Carne de Res,Aila,HOMBRE,18-24,C3,81.0
77,https://www.cav.cl/tienda/producto/aila-heaven...,12785,28846,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,Aila,Tinto,Cabernet Sauvignon,9990,91,"Rojo violeta brillante, de ribetes rubíes. Aro...",...,1.0,1.0,0.4,0.4,Carne de Res,Aila,HOMBRE,18-24,D/E,63.0
78,https://www.cav.cl/tienda/producto/aila-heaven...,12785,28846,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,Aila,Tinto,Cabernet Sauvignon,9990,91,"Rojo violeta brillante, de ribetes rubíes. Aro...",...,1.0,1.0,0.4,0.4,Carne de Res,Aila,HOMBRE,25-34,C2,61.0
79,https://www.cav.cl/tienda/producto/aila-heaven...,12785,28846,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,Aila,Tinto,Cabernet Sauvignon,9990,91,"Rojo violeta brillante, de ribetes rubíes. Aro...",...,1.0,1.0,0.4,0.4,Carne de Res,Aila,HOMBRE,25-34,C3,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43338,https://www.cav.cl/tienda/producto/vistamar-ca...,11663,27364,Vistamar Cabernet Sauvignon Gran Reserva 2018,Vistamar,Tinto,Cabernet Sauvignon,14990,90,"Rojo violeta profundo. Nariz perfumada, ciruel...",...,0.8,0.4,0.8,0.6,Cheddar,Vistamar,HOMBRE,55-75,C2,49.0
43339,https://www.cav.cl/tienda/producto/vistamar-ca...,11663,27364,Vistamar Cabernet Sauvignon Gran Reserva 2018,Vistamar,Tinto,Cabernet Sauvignon,14990,90,"Rojo violeta profundo. Nariz perfumada, ciruel...",...,0.8,0.4,0.8,0.6,Cheddar,Vistamar,MUJER,25-34,C2,99.0
43340,https://www.cav.cl/tienda/producto/vistamar-ca...,11663,27364,Vistamar Cabernet Sauvignon Gran Reserva 2018,Vistamar,Tinto,Cabernet Sauvignon,14990,90,"Rojo violeta profundo. Nariz perfumada, ciruel...",...,0.8,0.4,0.8,0.6,Cheddar,Vistamar,MUJER,25-34,D/E,114.0
43341,https://www.cav.cl/tienda/producto/vistamar-ca...,11663,27364,Vistamar Cabernet Sauvignon Gran Reserva 2018,Vistamar,Tinto,Cabernet Sauvignon,14990,90,"Rojo violeta profundo. Nariz perfumada, ciruel...",...,0.8,0.4,0.8,0.6,Cheddar,Vistamar,MUJER,35-44,C2,106.0


In [14]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# Suponiendo que 'df' es tu DataFrame
# Seleccionar solo las columnas relevantes
df = df[['product_name', 'price', 'pairing', 'gender', 'age', 'ses', 'rank']]

# Convertir la columna 'age' en un valor numérico y manejar posibles NaN
def convert_age_range(age_range):
    if pd.isna(age_range):
        return None
    elif '-' in str(age_range):
        ages = str(age_range).split('-')
        return (int(ages[0]) + int(ages[1])) / 2
    else:
        return float(age_range)

df['age'] = df['age'].apply(convert_age_range)
df['pairing'] = df['pairing'].str.strip()

# Manejar valores faltantes (NaN)
# Puedes elegir cómo manejar estos valores: eliminarlos o reemplazarlos
df.dropna(inplace=True)  # O usar df.fillna() con un valor o método específico

# Codificación one-hot para variables categóricas
one_hot_encoder = OneHotEncoder(sparse=False)
categorical_columns = ['product_name','gender', 'ses']
df_encoded = pd.DataFrame(one_hot_encoder.fit_transform(df[categorical_columns]))
df_encoded.columns = one_hot_encoder.get_feature_names_out(categorical_columns)

# Eliminar columnas originales categóricas y añadir las codificadas
df = df.drop(categorical_columns, axis=1)
df = pd.concat([df, df_encoded], axis=1)

# Normalización de variables numéricas
scaler = MinMaxScaler()
numeric_columns = ['price', 'rank', 'age']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

/tmp/ipykernel_7417/819493579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].apply(convert_age_range)
/tmp/ipykernel_7417/819493579.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pairing'] = df['pairing'].str.strip()
/tmp/ipykernel_7417/819493579.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [15]:
df = df.dropna()

In [299]:
df1 = df
df2 = df

In [16]:


# Preparar el conjunto de datos para el modelo k-NN
X = df.drop(['pairing'], axis=1)  # Excluye 'product_name' para la entrada del modelo

# Crear el modelo k-NN
knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)

# Función para obtener recomendaciones basadas en maridaje
def get_common_products_for_pairings(pairing_inputs):
    product_to_columns = {}  # Mapeo de nombre de producto a columnas codificadas
    for col in df_encoded.columns:
        if col.startswith('product_name_'):
            product_name = col.split('product_name_')[1]
            product_to_columns[product_name] = col

    # Filtrar las filas que coincidan con cada maridaje
    filtered_dfs = [df[df['pairing'] == pairing] for pairing in pairing_inputs if pairing in df['pairing'].unique()]

    if len(filtered_dfs) != len(pairing_inputs):
        missing_pairings = set(pairing_inputs) - set(df['pairing'].unique())
        print(f"Maridajes no encontrados: {missing_pairings}")
        return []

    # Identificar los productos comunes que aparecen en todas las listas de maridajes
    common_products = set(product_to_columns.keys())
    for filtered_df in filtered_dfs:
        current_products = set()
        for index, row in filtered_df.iterrows():
            for product, col in product_to_columns.items():
                if row[col] == 1:
                    current_products.add(product)
        common_products.intersection_update(current_products)

    if not common_products:
        return "No hay vinos que coincidan con todos los maridajes seleccionados."

    return common_products





In [17]:
X

,price,age,rank,product_name_Aila Heaven Cabernet Sauvignon Gran Reserva 2021,product_name_Baron Philippe De Rothschild Escudo Rojo Origine Cabernet Sauvignon 2019,product_name_Baron Phillippe De Rothschild Mapu Chardonnay 2022,product_name_Caiti Lux Series Sauvignon Blanc 2020,product_name_Choro Byron Pais 2020,product_name_Descomplicados Etiqueta Negra Syrah 2019,product_name_Longavi Sauvignon Blanc 2015,...,product_name_Vistamar Cabernet Sauvignon Gran Reserva 2018,product_name_Viña Andes Plateau Andesita Cabernet Sauvignon 2021,product_name_Viña Villalobos Nativa Pais 2020,gender_HOMBRE,gender_MUJER,gender_OTRO,ses_ABC1,ses_C2,ses_C3,ses_D/E
75,0.249875,0.12,0.073620,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
76,0.249875,0.12,0.478528,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
77,0.249875,0.12,0.368098,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
78,0.249875,0.29,0.355828,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
79,0.249875,0.29,0.380368,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5599,0.745127,0.49,0.779141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5600,0.745127,0.49,0.361963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5601,0.745127,0.49,0.638037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5602,0.745127,0.69,0.846626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# Luego, esta lista de productos comunes se puede usar para obtener recomendaciones.
pairing_inputs = ['Carne de Res', 'Comino']  # Lista de maridajes
common_products = get_common_products_for_pairings(pairing_inputs)
print(common_products)

{'Aila Heaven Cabernet Sauvignon Gran Reserva 2021', 'Longavi Sauvignon Blanc 2015'}


# 6. Test 2: Modelado: Input 1 meridaje, ses, gender y age

In [27]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
conn = sqlite3.connect('../data/wine_db.db')

# Leer tabla "wine_database"
query = "SELECT * FROM wine_database;"
df = pd.read_sql_query(query, conn)

conn.close()

In [28]:
df = df[['product_name', 'price', 'pairing', 'gender', 'age', 'ses', 'rank']]


In [29]:
# Paso 2: Preprocesamiento de los datos para el modelo k-NN

# Codificación One-Hot para variables categóricas
one_hot_encoder = OneHotEncoder(sparse=False)
categorical_columns = ['gender', 'ses', 'age']
df_encoded = pd.DataFrame(one_hot_encoder.fit_transform(df[categorical_columns]))
df_encoded.columns = one_hot_encoder.get_feature_names_out(categorical_columns)

# Normalización de variables numéricas
scaler = MinMaxScaler()
numeric_columns = ['price', 'rank']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Combinar las columnas codificadas y normalizadas
df = pd.concat([df, df_encoded], axis=1).drop(categorical_columns, axis=1)

/home/vscode/.local/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
df = df.dropna()

In [31]:
df_agrupado = df.groupby('product_name')['pairing'].apply(lambda x: ', '.join(set(x))).reset_index()

X = df.drop(['product_name', 'pairing'], axis=1)  # Excluir 'product_name' y 'pairing' para las características

# Entrenar el modelo k-NN
knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)

In [22]:
from joblib import dump

# Entrenar el modelo k-NN
knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)

# Guardar el modelo en un archivo
dump(knn, 'modelo_knn.joblib')

['modelo_knn.joblib']

In [32]:
df

,product_name,price,pairing,rank,gender_HOMBRE,gender_MUJER,gender_OTRO,gender_None,ses_ABC1,ses_C2,ses_C3,ses_D/E,ses_None,age_13-17,age_18-24,age_25-34,age_35-44,age_45-54,age_55-75,age_None
75,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,0.028570,Carne de Res,0.073034,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
76,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,0.028570,Carne de Res,0.443820,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
77,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,0.028570,Carne de Res,0.342697,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
78,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,0.028570,Carne de Res,0.331461,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
79,Aila Heaven Cabernet Sauvignon Gran Reserva 2021,0.028570,Carne de Res,0.353933,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44416,Vultur Globo Red Blend 2020,0.045712,Cheddar,0.601124,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
44417,Vultur Globo Red Blend 2020,0.045712,Cheddar,0.522472,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
44418,Vultur Globo Red Blend 2020,0.045712,Cheddar,0.662921,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
44419,Vultur Globo Red Blend 2020,0.045712,Cheddar,0.410112,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
def get_recommendations_by_pairings_gender_ses_and_age(pairing_inputs, gender_input, ses_input, age_input, k=5):
    # Identificar las columnas de género, SES y edad después del one-hot encoding
    gender_column = f'gender_{gender_input}'
    ses_column = f'ses_{ses_input}'
    age_column = f'age_{age_input}'

    # Verificar si las columnas existen
    if gender_column not in df.columns or ses_column not in df.columns or age_column not in df.columns:
        return "Género, SES o rango de edad especificado no disponible en los datos."

    # Filtrar vinos que coincidan con todos los maridajes de entrada, género, SES y rango de edad
    filtered_df = df[(df[gender_column] == 1) & (df[ses_column] == 1) & (df[age_column] == 1)]
    filtered_df = filtered_df[filtered_df['pairing'].apply(lambda x: all(pairing in x for pairing in pairing_inputs))]

    if filtered_df.empty:
        return []

    recommendations = []
    for product_name in filtered_df['product_name'].unique():
        # Obtener las características correspondientes al producto en X
        product_features = filtered_df[filtered_df['product_name'] == product_name].drop(['product_name', 'pairing'], axis=1)

        if not product_features.empty:
            # Utilizar la media de las características si hay múltiples filas para el mismo producto
            product_feature_avg = product_features.mean().values.reshape(1, -1)

            # Obtener recomendaciones utilizando k-NN
            distances, indices = knn.kneighbors(product_feature_avg, n_neighbors=k)
            for idx in indices[0]:
                recommended_product_name = df.iloc[idx]['product_name']
                recommendations.append(recommended_product_name)

    return list(set(recommendations))  # Eliminar duplicados

In [35]:
# Ejemplo de uso
pairing_inputs = ['Comino']  # Lista de maridajes
gender_input = 'MUJER'  # Género
ses_input = 'C3'   # SES
age_input = '35-44'  # Rango de Edad
recommendations = get_recommendations_by_pairings_gender_ses_and_age(pairing_inputs, gender_input, ses_input, age_input)
print(recommendations)

[]


# 6. Test 3: Modelado:  Varios input + pairing (varios) + gender + ses + age

In [65]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
conn = sqlite3.connect('../data/wine_db.db')

# Leer tabla "wine_database"
query = "SELECT * FROM wine_database;"
df = pd.read_sql_query(query, conn)

conn.close()

In [66]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors


# Filtrando las columnas relevantes
df = df[['product_name', 'price', 'pairing', 'gender', 'age', 'ses', 'rank', 'image_url']]

# Manejo de valores faltantes
imputer = SimpleImputer(strategy='most_frequent')
df[['gender', 'age', 'ses', 'rank']] = imputer.fit_transform(df[['gender', 'age', 'ses', 'rank']])

# Codificación One-Hot para variables categóricas
one_hot_encoder = OneHotEncoder(sparse=False)
categorical_columns = ['gender', 'ses', 'age']
df_encoded = pd.DataFrame(one_hot_encoder.fit_transform(df[categorical_columns]))
df_encoded.columns = one_hot_encoder.get_feature_names_out(categorical_columns)

# Normalización de variables numéricas
scaler = MinMaxScaler()
numeric_columns = ['price', 'rank']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Combinar las columnas codificadas y normalizadas
df = pd.concat([df, df_encoded], axis=1).drop(categorical_columns, axis=1)

# Limpieza final
df = df.dropna()

# Agrupar datos por nombre de producto y maridajes
df_agrupado = df.groupby('product_name')['pairing'].apply(lambda x: ', '.join(set(x))).reset_index()


/home/vscode/.local/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [67]:
df

,product_name,price,pairing,rank,gender_HOMBRE,gender_MUJER,gender_OTRO,gender_None,ses_ABC1,ses_C2,ses_C3,ses_D/E,ses_None,age_13-17,age_18-24,age_25-34,age_35-44,age_45-54,age_55-75,age_None
0,Miguel Torres Almado Ensamblaje Tinto 2018,0.090852,Carne de Res,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Miguel Torres Almado Ensamblaje Tinto 2018,0.090852,Cilantro,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Miguel Torres Almado Ensamblaje Tinto 2018,0.090852,Colby,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Miguel Torres Almado Ensamblaje Tinto 2018,0.090852,Comino,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Miguel Torres Almado Ensamblaje Tinto 2018,0.090852,Cordero,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44495,William Fevre Espino Reserva Especial Cabernet...,0.022856,Lenteja,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
44496,William Fevre Espino Reserva Especial Cabernet...,0.022856,Muenster,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
44497,William Fevre Espino Reserva Especial Cabernet...,0.022856,Ossau-Iraty,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
44498,William Fevre Espino Reserva Especial Cabernet...,0.022856,Venado,0.38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Asumiendo que df es tu DataFrame y 'label' es el nombre de la columna de etiquetas
# Separar características y etiquetas
X = df.drop('pairing', axis=1)  # Todas las columnas excepto la de etiquetas
y = df['pairing']  # Columna de etiquetas

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = knn.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

ValueError: could not convert string to float: 'Valdivieso Single Vineyard Sauvignon Blanc 2018'

In [68]:
def get_recommendations_by_pairings_gender_and_ses(pairing_inputs, gender_input, ses_input, age_input, k=5):
    # Identificar las columnas de género y SES después del one-hot encoding
    gender_column = f'gender_{gender_input}'
    ses_column = f'ses_{ses_input}'
    age_column = f'age_{age_input}'


    # Verificar si las columnas existen
    if gender_column not in df.columns or ses_column not in df.columns or age_column not in df.columns:
        return "Género, Rango etario o NSE especificado no se encuentra disponible en los datos."

    # Pre-filtrado basado en maridajes, género y SES
    df_filtrado = df[df['product_name'].isin(df_agrupado[df_agrupado['pairing'].apply(lambda x: all(pairing in x for pairing in pairing_inputs))]['product_name'])]
    df_filtrado = df_filtrado[(df_filtrado[gender_column] == 1) & (df_filtrado[ses_column] == 1) & (df_filtrado[age_column] == 1)]

    if df_filtrado.empty:
        return []

    # Inicializar el modelo k-NN
    knn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(df_filtrado.drop(['product_name', 'pairing'], axis=1))

    recommendations = []
    for product_name in df_filtrado['product_name'].unique():
        # Obtener las características del producto
        product_features = df_filtrado[df_filtrado['product_name'] == product_name].drop(['product_name', 'pairing'], axis=1)

        if not product_features.empty:
            # Utilizar la media de las características si hay múltiples filas para el mismo producto
            product_feature_avg = product_features.mean().values.reshape(1, -1)

            # Obtener recomendaciones utilizando k-NN
            distances, indices = knn.kneighbors(product_feature_avg, n_neighbors=k)
            for idx in indices[0]:
                recommended_product_name = df_filtrado.iloc[idx]['product_name']
                recommendations.append(recommended_product_name)

    return list(set(recommendations))  # Eliminar duplicados


In [70]:
pairing_inputs = ['Carne de Res', 'Comino', 'Cheddar']  # Lista de maridajes
gender_input = 'MUJER'  # Género
age_input = '25-34'   # AGE
ses_input = 'ABC1'   # SES
recommendations = get_recommendations_by_pairings_gender_and_ses(pairing_inputs, gender_input, ses_input, age_input)
print(recommendations)


['Viña 3 Monos Carignan/Garnacha Mirador Del Name Cauquenes 2017', 'Vultur Globo Red Blend 2020', 'Aila Heaven Cabernet Sauvignon Gran Reserva 2021', 'Calyptra Vivendo Ensamblaje Tinto 2017', 'El Principal Calicanto Cabernet Sauvignon / Carmenere 2021', 'Catena Malbec 2020', 'Choro Byron Pais 2020', 'Villard Expresion Syrah 2020', 'Viña 3 Monos Cabernet Sauvignon 2019', 'Viña Trivento Golden Reserve Malbec 2020', 'Jose Cabernet Sauvignon 2019', 'Rocas De Seña Ensamblaje Tinto 2020', 'Argento Single Block Malbec 2019', 'Ineditas El Tartufo Malbec 2015', 'Salentein Malbec Reserve 2022']


/home/vscode/.local/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/bas

In [64]:
from joblib import dump

dump(knn, '../models/wine_pairing_recommendation_knn.joblib')

['../models/wine_pairing_recommendation_knn.joblib']

# 7. Despliegue